In [1]:
import os
import numpy as np
# import tensorflow as tf
import cv2
import random
import seaborn as sns
import matplotlib.pyplot as plt
 
# from keras.models import Sequential, Model
# from keras.layers import Dense, Dropout, Activation, Flatten, Input
# from keras.layers.convolutional import Conv2D, MaxPooling2D
# from keras.optimizers import RMSprop, Adam, SGD
# from keras.preprocessing import image
# from keras.preprocessing.image import ImageDataGenerator
# from keras.utils import np_utils
# from keras.applications.vgg16 import VGG16, preprocess_input
# from sklearn.model_selection import train_test_split

In [2]:
file_dir_path='/media/aaron/新加卷1/datasetLVST/train_full_images_0'
file_path='/media/aaron/新加卷1/datasetLVST/train_full_labels.json'

In [3]:
import json

from PIL import Image, ImageDraw, ImageFont

In [4]:
#注意，json模块需要读取的文件对象，给他路径名会报错。。。
with open(file_path) as jsonfile:
    data = json.load(jsonfile)

In [6]:
for item in data['gt_0']:
    print (item)

{'points': [[595, 582], [621, 598], [619, 620], [593, 607]], 'illegibility': True, 'transcription': '###'}
{'points': [[232, 223], [321, 200], [326, 563], [229, 571]], 'illegibility': False, 'transcription': '高尔夫'}
{'points': [[329, 191], [366, 184], [368, 559], [332, 564]], 'illegibility': False, 'transcription': '聚鑫惠通虚拟现实'}
{'points': [[213, 577], [387, 564], [389, 597], [212, 609]], 'illegibility': False, 'transcription': 'USG 教学工作室'}
{'points': [[217, 612], [379, 601], [379, 629], [217, 638]], 'illegibility': False, 'transcription': '高碑店东区166号'}
{'points': [[554, 521], [597, 551], [594, 608], [550, 585]], 'illegibility': False, 'transcription': '连接'}


In [ ]:
index=50
img=cv2.imread(os.path.join(file_dir_path,'gt_{}.jpg'.format(index)))
for item in data['gt_{}'.format(index)]:
    c = (0,255,0)
    cv2.line(img,(item['points'][0][0],item['points'][0][1]),(item['points'][1][0],item['points'][1][1]),c,2)
    cv2.line(img,(item['points'][1][0],item['points'][1][1]),(item['points'][2][0],item['points'][2][1]),c,2)
    cv2.line(img,(item['points'][2][0],item['points'][2][1]),(item['points'][3][0],item['points'][3][1]),c,2)
    cv2.line(img,(item['points'][3][0],item['points'][3][1]),(item['points'][0][0],item['points'][0][1]),c,2)
#     cv2.putText(img,item['transcription'],(item['points'][0][0],item['points'][0][1]), font, 2,(255,255,255),3)

img_PIL = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
font = ImageFont.truetype('NotoSansCJK-Black.ttc', 40)
draw = ImageDraw.Draw(img_PIL)

for item in data['gt_{}'.format(index)]:    
    draw.text((item['points'][0][0],item['points'][0][1]), item['transcription'], font=font, fill=(255,255,255))
    # 使用PIL中的save方法保存图片到本地
    # img_PIL.save('02.jpg', 'jpeg')
 
    # 转换回OpenCV格式
img = cv2.cvtColor(np.asarray(img_PIL),cv2.COLOR_RGB2BGR)
cv2.namedWindow('ts', cv2.WINDOW_AUTOSIZE)
cv2.imshow('ts',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

__可以考虑循环这个标签，然后找到是库里的那些标签，然后把他都裁剪出来，然后做数据增强，然后和我们的数据放到一块去训练。这样我们的模型应该是可以提高很多。
同时，关于数据增强的部分，通过放大，然后裁剪也是必要的
注意关于颜色的选择很值得思考__

__在这个场景文字识别的部分来看，我还是倾向于有效的切割是更为有效的方法，通过大量端对端的学习并非不可，只是对于千万种组合而言，是在是有点难以控制，特别是对于我们这种小的公司而言，可以在后面搞一个文字联想连接的网络，输入几个字来联想成有效的组合，这种组合应该在实际使用场景中是有限的，就像词库一样，这样，分割过程中的识别问题是有可能得到有效的解决的。另外，感觉这种方法也和人类的认知过程比较接近吧，人类也都是去大概的认识一下，然后凭借经验去猜测是一个什么内容，在不同场景下的猜测自然也不同。用给的环境数据建立联想库，然后分割文本，分别识别，吧有瑕疵的文本识别组合起来，输入得到一个该场景下的有效结果。__

In [32]:
# -*- coding: utf-8 -*-
import cv2
import numpy
from PIL import Image, ImageDraw, ImageFont
 
if __name__ == '__main__':
 
    img_OpenCV = cv2.imread('01.jpg')
    # 图像从OpenCV格式转换成PIL格式
    img_PIL = Image.fromarray(cv2.cvtColor(img_OpenCV, cv2.COLOR_BGR2RGB))
 
    # 字体  字体*.ttc的存放路径一般是： /usr/share/fonts/opentype/noto/ 查找指令locate *.ttc
    font = ImageFont.truetype('NotoSansCJK-Black.ttc', 40)
    # 字体颜色
    fillColor = (255,0,0)
    # 文字输出位置
    position = (100,100)
    # 输出内容
    str = '在图片上输出中文'
 
    # 需要先把输出的中文字符转换成Unicode编码形式
    if not isinstance(str, unicode):
        str = str.decode('utf8')
 
    draw = ImageDraw.Draw(img_PIL)
    draw.text(position, str, font=font, fill=fillColor)
    # 使用PIL中的save方法保存图片到本地
    # img_PIL.save('02.jpg', 'jpeg')
 
    # 转换回OpenCV格式
    img_OpenCV = cv2.cvtColor(numpy.asarray(img_PIL),cv2.COLOR_RGB2BGR)
    cv2.imshow("print chinese to image",img_OpenCV)
    cv2.waitKey()
    cv2.imwrite('03.jpg',img_OpenCV)
 